In [ ]:
#requirements
import csv
import numpy
import psycopg2
from datetime import datetime
import requests
import pandas as pd
import folium
import ipywidgets as widgets
import urllib.parse
import branca.colormap as cm
import json

states = ['---', "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

conn = None
curr = None
try:
    conn = psycopg2.connect("dbname=********* user=******* password=*********")
    print('ready to go!')
except:
    print("no connection!")

In [ ]:
def make_the_map():
    scale = scale_widget.value
    args=subset_widget.value

    scalar = {"region":"FAC_EPA_REGION", "state": "FAC_STATE", "district": "FAC_DERIVED_CD113", 
              "zip": "FAC_ZIP", "watershed": "FAC_DERIVED_HUC"}

    if (scale == "state" or scale =="region"):
        sql="SELECT t1."+scalar[scale]+", noncompliant, total "+\
        "FROM("+\
        "SELECT "+scalar[scale]+", COUNT(REGISTRY_ID) AS noncompliant FROM ECHO_EXPORTER WHERE (FAC_COMPLIANCE_STATUS like '%Significant%' or FAC_COMPLIANCE_STATUS = 'Violation Identified') GROUP BY "+scalar[scale]+") t1 " +\
        "INNER JOIN"+\
        "(SELECT "+scalar[scale]+", COUNT(REGISTRY_ID) AS total FROM ECHO_EXPORTER GROUP BY "+scalar[scale]+") t2 "+\
        "ON (t1."+scalar[scale]+" = t2."+scalar[scale]+")"
        print("Ok!")

    elif (scale == "district" or scale =="zip" or scale=="watershed") and len(args)==2:
        sql="SELECT t1."+scalar[scale]+", noncompliant, total "+\
        "FROM("+\
        "SELECT "+scalar[scale]+", COUNT(REGISTRY_ID) AS noncompliant FROM ECHO_EXPORTER WHERE FAC_STATE='"+args+"' and (FAC_COMPLIANCE_STATUS like '%Significant%' or FAC_COMPLIANCE_STATUS = 'Violation Identified') GROUP BY "+scalar[scale]+") t1 " +\
        "INNER JOIN"+\
        "(SELECT "+scalar[scale]+", COUNT(REGISTRY_ID) AS total FROM ECHO_EXPORTER WHERE FAC_STATE='"+args+"' GROUP BY "+scalar[scale]+") t2 "+\
        "ON (t1."+scalar[scale]+" = t2."+scalar[scale]+")"
        print("Ok!")

    else:
        print("Please pick a specific state first.")
        
    # edit sql queries to: where compliance status = significant violation
    
    url='http://apps.tlt.stonybrook.edu/echoepa/?query='
    data_location=url+urllib.parse.quote(sql)
    print("Getting the data on non-compliance!")
    if (scale == "zip"):
        att_data = pd.read_csv(data_location, encoding='iso-8859-1', header = 0, dtype={scalar[scale]:"str"}) # don't conver to integer!
    else:
        att_data = pd.read_csv(data_location, encoding='iso-8859-1', header = 0)
    att_data["percent"]=(att_data["noncompliant"]/att_data["total"])*100
    att_data = att_data.rename(columns={scalar[scale]: "id"})

    # print(att_data) # printing enables scroll bars
    
    # Get geography data from my server or elsewhere.
    if (scale=="state"):
        url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json'
        geo_json_data = json.loads(requests.get(url).text)

    else:
        if (scale=="region"):
            sql = "SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features " + \
                    "FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, number As id, row_to_json((number, EPAREGION)) as properties " + \
                    "FROM "+scale+"s As lg) As f) As fc"; # add s

        if (scale == "district"):
            sql = "SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features " + \
                    "FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, number As id, row_to_json((number, STUSPS)) as properties " + \
                    "FROM "+scale+"s As lg WHERE lg.STUSPS='"+args+"') As f) As fc"; # add s

        if (scale == "zip"):
            att_data['id']= att_data['id'].astype(str)
            sql = "SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features " + \
                    "FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, zcta5ce10 As id, row_to_json((zcta5ce10, STUSPS)) As properties " + \
                    "FROM "+scale+"s As lg WHERE lg.STUSPS='"+args+"') As f) As fc"; # add s
    
        if (scale == "watershed"):
            att_data['id']= att_data['id'].astype(str)
            sql = "SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features " + \
                    "FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, huc8 As id, row_to_json((huc8, states)) As properties " + \
                    "FROM "+scale+" As lg WHERE lg.states like '%"+args+"%') As f) As fc";
            
        #make above more efficient
        print("Getting the geographic data...")

        try: 
            curr = conn.cursor()
            curr.execute(sql)
            geo_json_data = curr.fetchall()
            geo_json_data = geo_json_data[0][0] # Turns DB files into proper GeoJSONs
            print("Got it!")
        except:
            print("Well, that didn't work")
    
    print("Making the map!")
    q = att_data['percent'].quantile([0, 0.25,0.5,0.75, 1])

    m = folium.Map(location=[40, -105], zoom_start=4)
    c = folium.Choropleth(
        geo_data = geo_json_data,
        data = att_data,
        columns=['id', 'percent'], key_on='feature.id',
        fill_color='OrRd',fill_opacity=0.75,line_weight=.5,nan_fill_opacity=.1, nan_fill_color="white", highlight=True,
        bins=[q[0.00],q[0.25], q[0.5], q[0.75], q[1.00]],
        legend_name="Rate of Non-Compliance"
    ).add_to(m)

    if (scale is not "state"):
        c.geojson.add_child(
            folium.features.GeoJsonTooltip(['f1'])
        )

    bounds = m.get_bounds() # Does not work well for the states layer, because of overseas territories
    m.fit_bounds(bounds)

    return m

In [ ]:
print("If you view by congressional district, watershed, or zip code, please use the second menu (Zoom in on) to pick a specific state")
scale_widget=widgets.Dropdown(
    options=['region', 'state', 'district', 'watershed', 'zip'], 
    description='Map by:',
    disabled=False,
    )
display(scale_widget)
subset_widget=widgets.Dropdown(
    options=states, 
    description='Zoom in on:',
    disabled=False,
    )
display(subset_widget)

In [ ]:
make_the_map()

In [ ]:
# Close our connections
curr.close()
conn.close()